In [1]:
%matplotlib inline
import importlib
import utils2; importlib.reload(utils2)
from utils2 import *

Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
def res_block(inp, kernel_size): 
    num_outputs = inp.get_shape()[-1].value
    h1 = Convolution2D(nb_filter=num_outputs, nb_row=kernel_size, nb_col=kernel_size, 
                      subsample=1)

In [2]:
x = Input(shape=(220, 220, 3))
#TODO: Mirror padding instead of zero-padding; use old module for that 
x = Convolution2D(nb_filter=32, nb_row=9, nb_col=9, subsample=1, border_mode='same')(x)
x = Convolution2D(nb_filter=64, nb_row=3, nb_col=3, subsample=2, border_mode='same')(x)
x = Convolution2D(nb_filter=128, nb_row=3, nb_col=3, subsample=2, border_mode='same')(x)


In [6]:
x.get_shape()[-1].value

3

In [ ]:
#TODO: function to do conditional normalization. I think here it's just as simple as passing in a
#an additional parameter when your'e training 

In [1]:
import tensorflow as tf

slim = tf.contrib.slim

In [2]:
sess = tf.InteractiveSession()
x = tf.Variable([1.0, 2.0])

In [37]:
style_labels = tf.constant([[0, 1], [6, 7], [9, 5]])

In [38]:
style_labels.shape

TensorShape([Dimension(3), Dimension(2)])

In [39]:
style_labels.eval()

array([[0, 1],
       [6, 7],
       [9, 5]], dtype=int32)

In [ ]:
style_labels = []

In [ ]:
tf.nn.batch_normalization()

In [26]:
tf.expand_dims(tf.expand_dims(foo, 1), 1)

<tf.Tensor 'ExpandDims_6:0' shape=(2, 1, 1, 3) dtype=int32>

In [40]:
shape = tf.TensorShape([4]).concatenate(6)

In [41]:
shape

TensorShape([Dimension(4), Dimension(6)])

In [43]:
beta = slim.model_variable('beta2',
                                shape=shape,
                                dtype='float64',
                                initializer=tf.zeros_initializer())

In [50]:
tf.global_variables_initializer().run()

In [53]:
style_labels = tf.constant([0, 0, 3, 0, 1, 2])

In [ ]:
tf.gather()

In [54]:
tf.gather(beta, style_labels).eval()

array([[ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.]])

In [ ]:
shape = tf.TensorShape([num_categories]).concatenate(params_shape)
var_collections = slim.utils.get_variable_collections(
  variables_collections, name)
var = slim.model_variable(name,
                        shape=shape,
                        dtype=dtype,
                        initializer=initializer,
                        collections=var_collections,
                        trainable=trainable)
conditioned_var = tf.gather(var, labels)
conditioned_var = tf.expand_dims(tf.expand_dims(conditioned_var, 1), 1)
return conditioned_var

In [1]:
from keras.layers import Input

Using TensorFlow backend.


In [11]:
from keras.layers import merge

In [6]:
inp = Input(shape=(3,))

[<tf.Tensor 'input_2:0' shape=(?, 3) dtype=float32>,
 <tf.Tensor 'input_2:0' shape=(?, 3) dtype=float32>,
 <tf.Tensor 'input_2:0' shape=(?, 3) dtype=float32>,
 <tf.Tensor 'input_2:0' shape=(?, 3) dtype=float32>,
 <tf.Tensor 'input_2:0' shape=(?, 3) dtype=float32>,
 <tf.Tensor 'input_2:0' shape=(?, 3) dtype=float32>,
 <tf.Tensor 'input_2:0' shape=(?, 3) dtype=float32>,
 <tf.Tensor 'input_2:0' shape=(?, 3) dtype=float32>,
 <tf.Tensor 'input_2:0' shape=(?, 3) dtype=float32>,
 <tf.Tensor 'input_2:0' shape=(?, 3) dtype=float32>]

In [14]:
x = merge([inp for i in range(10)], mode='concat')

In [15]:
from keras import backend as K 

In [16]:
K.reshape(x, shape=(10, 3))

<tf.Tensor 'Reshape:0' shape=(10, 3) dtype=float32>

## Conditional Instance Normalization Layer

In [ ]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import

from ..engine import Layer, InputSpec
from .. import initializers
from .. import regularizers
from .. import constraints
from .. import backend as K


class ConditionalInstanceNormalization(Layer):
    """Builds upon (i.e. borrows liberally from) the existing Batch 
    Normalization layer within Keras, but then implements Conditional Instance Normalization
    as described here: https://arxiv.org/pdf/1610.07629.pdf
    # Arguments
        num_categories: The number of categories upon which you're conditionally normalizing. 
                        If using for style transfer, this corresponds to the
                        number of distinct styles. Set to 2 by default. 
                        
        axis: Integer, the axis that should be normalized
            (typically the features axis).
            For instance, after a `Conv2D` layer with
            `data_format="channels_first"`,
            set `axis=1` in `BatchNormalization`.
                    
        momentum: Momentum for the moving average.
        epsilon: Small float added to variance to avoid dividing by zero.
        center: If True, add offset of `beta` to normalized tensor.
            If False, `beta` is ignored.
        scale: If True, multiply by `gamma`.
            If False, `gamma` is not used.
            When the next layer is linear (also e.g. `nn.relu`),
            this can be disabled since the scaling
            will be done by the next layer.
        beta_initializer: Initializer for the beta weight.
        gamma_initializer: Initializer for the gamma weight.
        moving_mean_initializer: Initializer for the moving mean.
        moving_variance_initializer: Initializer for the moving variance.
        beta_regularizer: Optional regularizer for the beta weight.
        gamma_regularizer: Optional regularizer for the gamma weight.
        beta_constraint: Optional constraint for the beta weight.
        gamma_constraint: Optional constraint for the gamma weight.
    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
    # Output shape
        Same shape as input.
    # References
TODO 
"""

    def __init__(self,
                 num_categories=2
                 axis=-1,
                 momentum=0.99,
                 epsilon=1e-3,
                 center=True,
                 scale=True,
                 beta_initializer='zeros',
                 gamma_initializer='ones',
                 moving_mean_initializer='zeros',
                 moving_variance_initializer='ones',
                 beta_regularizer=None,
                 gamma_regularizer=None,
                 beta_constraint=None,
                 gamma_constraint=None,
                 **kwargs):
        super(ConditionalInstanceNormalization, self).__init__(**kwargs)
        self.supports_masking = True
        self.num_categories = num_categories 
        self.axis = axis
        self.momentum = momentum
        self.epsilon = epsilon
        self.center = center
        self.scale = scale
        self.beta_initializer = initializers.get(beta_initializer)
        self.gamma_initializer = initializers.get(gamma_initializer)
        self.moving_mean_initializer = initializers.get(moving_mean_initializer)
        self.moving_variance_initializer = initializers.get(moving_variance_initializer)
        self.beta_regularizer = regularizers.get(beta_regularizer)
        self.gamma_regularizer = regularizers.get(gamma_regularizer)
        self.beta_constraint = constraints.get(beta_constraint)
        self.gamma_constraint = constraints.get(gamma_constraint)

    def build(self, input_shape):
        dim = input_shape[self.axis]
        if dim is None:
            raise ValueError('Axis ' + str(self.axis) + ' of '
                             'input tensor should have a defined dimension '
                             'but the layer received an input with shape ' +
                             str(input_shape) + '.')
        self.input_spec = InputSpec(ndim=len(input_shape),
                                    axes={self.axis: dim})
        shape = (dim,)
        

        if self.scale:
            self.gamma = [self.add_weight(shape,
                                         name='gamma',
                                         initializer=self.gamma_initializer,
                                         regularizer=self.gamma_regularizer,
                                         constraint=self.gamma_constraint) for i in range(self.num_categories)]
         
        else:
            self.gamma = None
        if self.center:
            self.beta = [self.add_weight(shape,
                                        name='beta',
                                        initializer=self.beta_initializer,
                                        regularizer=self.beta_regularizer,
                                        constraint=self.beta_constraint) for i in range(self.num_categories)]

        else:
            self.beta = None
        self.moving_mean = self.add_weight(
            shape,
            name='moving_mean',
            initializer=self.moving_mean_initializer,
            trainable=False)
        self.moving_variance = self.add_weight(
            shape,
            name='moving_variance',
            initializer=self.moving_variance_initializer,
            trainable=False)
        self.built = True

    def call(self, inputs, training=None):
        activations = inputs[0]
        labels = inputs[1]
        #so, in practice, I think we can keep a lot of this and 
        # just call it on inputs[0], keeping inputs[1] as a way to lookup 
        input_shape = K.int_shape(inputs[0])
        
        # Prepare broadcasting shape.
        ndim = len(input_shape)
        reduction_axes = list(range(len(input_shape)))
        #[0, 1, 2, 3, 4]
        del reduction_axes[self.axis]
        #we delete the reduction axis along which we want to not marginalize 
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]
        #broadcast shape will be [1] for every axis other than the feature axis

        # Determines whether broadcasting is needed.
        needs_broadcasting = (sorted(reduction_axes) != list(range(ndim))[:-1])
        
        normed, mean, variance = K.normalize_batch_in_training(
            inputs[0], self.gamma, self.beta, reduction_axes,
            epsilon=self.epsilon)
        #okay, so the thing you want here is something with rows = len(batch), with each row 
        #being the oldschool equivalent of a gamma or beta 

        if training in {0, False}:
            return normed
        else:
            self.add_update([K.moving_average_update(self.moving_mean,
                                                     mean,
                                                     self.momentum),
                             K.moving_average_update(self.moving_variance,
                                                     variance,
                                                     self.momentum)],
                            inputs)

            def normalize_inference():
                if needs_broadcasting:
                    # In this case we must explictly broadcast all parameters.
                    broadcast_moving_mean = K.reshape(self.moving_mean,
                                                      broadcast_shape)
                    broadcast_moving_variance = K.reshape(self.moving_variance,
                                                          broadcast_shape)
                    if self.center:
                        broadcast_beta = K.reshape(self.beta, broadcast_shape)
                    else:
                        broadcast_beta = None
                    if self.scale:
                        broadcast_gamma = K.reshape(self.gamma,
                                                    broadcast_shape)
                    else:
                        broadcast_gamma = None
                    return K.batch_normalization(
                        inputs,
                        broadcast_moving_mean,
                        broadcast_moving_variance,
                        broadcast_beta,
                        broadcast_gamma,
                        epsilon=self.epsilon)
                else:
                    return K.batch_normalization(
                        inputs,
                        self.moving_mean,
                        self.moving_variance,
                        self.beta,
                        self.gamma,
                        epsilon=self.epsilon)

        # Pick the normalized form corresponding to the training phase.
        return K.in_train_phase(normed,
                                normalize_inference,
                                training=training)

    def get_config(self):
        config = {
            'axis': self.axis,
            'momentum': self.momentum,
            'epsilon': self.epsilon,
            'center': self.center,
            'scale': self.scale,
            'beta_initializer': initializers.serialize(self.beta_initializer),
            'gamma_initializer': initializers.serialize(self.gamma_initializer),
            'moving_mean_initializer': initializers.serialize(self.moving_mean_initializer),
            'moving_variance_initializer': initializers.serialize(self.moving_variance_initializer),
            'beta_regularizer': regularizers.serialize(self.beta_regularizer),
            'gamma_regularizer': regularizers.serialize(self.gamma_regularizer),
            'beta_constraint': constraints.serialize(self.beta_constraint),
            'gamma_constraint': constraints.serialize(self.gamma_constraint)
        }
        base_config = super(BatchNormalization, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
